# Advanced Experiments

This jupyter notebook file consists of more advanced and computationally expensive experiments in order to truly examine the results of using the different explanation methods. 

- Measure fidelity when edge mask is set to None to assess the importance of node features alone
    - Plot fidelity curves for various values of k (10, 20, 40, 80)
- Plot fidelity curves for subgraphs with various values of k (10, 20, 40, 80)
- Revised stability calculations
    - Can we do 5  runs for each of 10  instances?
    - Calculate average Jaccard similarity for each instance
    - Make a box plot or something similar to show distribution of similarity measures across instances
    - Per Yuriy’s suggestion, we could also calculate Ruzicka similarity (https://en.wikipedia.org/wiki/Jaccard_index) on soft masks
- Additional explainer methods
    - DummyExplainer (random baseline)
    - CaptumExplainer Saliency (not essential, but would be nice to include)
- Conduct evaluation on another GNN
    - EPGAT maybe?  (https://pubmed.ncbi.nlm.nih.gov/33497339/)


In [13]:
# install necessary packages

%pip install torch_geometric # install pytorch geometric
%pip install torchvision #install torchvision
%pip install matplotlib #install matplotlib
%pip install graphviz # install graphviz

%matplotlib inline

from torch_geometric.data import Data, DataLoader
from torch_geometric.explain import GNNExplainer,Explainer,GraphMaskExplainer,PGExplainer, DummyExplainer
import torch_geometric.transforms as T
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATv2Conv
from torch.nn import Linear,Softmax
import os
from tqdm import tqdm, trange
import pickle
from torch_geometric.explain.metric import fidelity, characterization_score
import json
import time
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
class SimpleGAT_1(torch.nn.Module):
    """
    A graph attention network with 4 graph layers and 2 linear layers.
    Uses v2 of graph attention that provides dynamic instead of static attention.
    The graph layer dimension and number of attention heads can be specified.
    """
    #torch.device('mps')
    def __init__(self, dataset, dim=8, num_heads=4):
        super(SimpleGAT_1, self).__init__()
        torch.manual_seed(seed=123)
        self.conv1 = GATv2Conv(in_channels=dataset.num_features, out_channels=dim, heads=num_heads,edge_dim=dataset.edge_attr.shape[1])
        self.conv2 = GATv2Conv(in_channels=dim * num_heads, out_channels=dim, heads=num_heads,edge_dim=dataset.edge_attr.shape[1])
        self.lin1 = Linear(dim * num_heads,dim)
        self.lin2 = Linear(dim,1)

    def forward(self, x, edge_index,edge_attr):
        h = self.conv1(x, edge_index,edge_attr).relu()
        h = self.conv2(h, edge_index,edge_attr).relu()
        h = self.lin1(h).relu()
        #print(h)
        h = F.dropout(h, p=0.1, training=self.training)
        out = self.lin2(h)[:,0]
        out = torch.sigmoid(out)
        return out

In [ ]:
data = torch.load('../new_SREBP2_2.pt')
model_path = '../SimpleGAT_1_model_lr5e-05_dp_0.0SREBP2.pth'
model = SimpleGAT_1(data,dim = 16)
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
model.eval()

In [4]:
with open('../feature_index.json') as json_file:
    feat_labels = json.load(json_file)

{'0': 'S',
 '1': 'ES',
 '2': 'TS',
 '3': 'DS',
 '4': 'EES',
 '5': 'ETS',
 '6': 'EDS',
 '7': 'TES',
 '8': 'TTS',
 '9': 'TDS',
 '10': 'DES',
 '11': 'DTS',
 '12': 'DDS',
 '13': 'EEES',
 '14': 'EETS',
 '15': 'EEDS',
 '16': 'ETES',
 '17': 'ETTS',
 '18': 'ETDS',
 '19': 'EDES',
 '20': 'EDTS',
 '21': 'EDDS',
 '22': 'TEES',
 '23': 'TETS',
 '24': 'TEDS',
 '25': 'TTES',
 '26': 'TTTS',
 '27': 'TTDS',
 '28': 'TDES',
 '29': 'TDTS',
 '30': 'TDDS',
 '31': 'DEES',
 '32': 'DETS',
 '33': 'DEDS',
 '34': 'DTES',
 '35': 'DTTS',
 '36': 'DTDS',
 '37': 'DDES',
 '38': 'DDTS',
 '39': 'DDDS',
 '40': 'has_path',
 '41': 'num_paths',
 '42': 'max_path_score',
 '43': 'max_degree_score',
 '44': 'thermo0',
 '45': 'thermo1',
 '46': 'thermo2',
 '47': 'thermo3',
 '48': 'thermo4',
 '49': 'thermo5',
 '50': 'thermo6',
 '51': 'RNA line ab',
 '52': 'binary_RNA line ab',
 '53': 'endosomes',
 '54': 'secretory',
 '55': 'vesicles',
 '56': 'cytoplasmic bodies',
 '57': 'mitochondria',
 '58': 'aggresome',
 '59': 'focal adhesion sites'

In [5]:
# extract node labels and testing nodes from node order 

""" 
# 9606.ENSP00000289989 lowest False Negative / predicted value: 0.0281679704785347 / node index: 17397
# 9606.ENSP00000358777 highest True Positive / predicted value: 0.9995648264884949 / node index: 9005
# 9606.ENSP00000216099 lowest True Negative / predicted value: 7.5380116e-30 / node index: 8665
# 9606.ENSP00000357637 highest False Positive / predicted value: 0.9998927 / node index: 12633
# 9606.ENSP00000357226 furthest to target with label 0 / predicted value: 1.4533093e-07 / node index: 18233
# 9606.ENSP00000344741 closest to target with label 1 / predicted value: 0.8364509 / node index: 12257
# 9606.ENSP00000470087 closest to target with label 0 / predicted value: 0.3407591 / node index: 1674
# 9606.ENSP00000289989 furthest to target with label 1 / predicted value: 0.02816797 / node index: 17397
# 9606.ENSP00000348069: SREBF1  / predicted value: 0.55870503 / node index: 6334
# 9606.ENSP00000370695: EXCO1 / predicted value: 0.50875914/ node index: 1883
"""

with open('../node_order.pickle', 'rb') as f:
    node_order = pickle.load(f)

node_id_labels = list(node_order.keys())

test_nodes = [node_order['9606.ENSP00000289989'], # lowest False Negative / predicted value: 0.0281679704785347 / node index: 17397
                node_order['9606.ENSP00000358777'], # highest True Positive / predicted value: 0.9995648264884949 / node index: 9005
                node_order['9606.ENSP00000216099'], # lowest True Negative / predicted value: 7.5380116e-30 / node index: 8665
                node_order['9606.ENSP00000357637'],  # highest False Positive / predicted value: 0.9998927 / node index: 12633
                node_order['9606.ENSP00000357226'], # furthest to target with label 0 / predicted value: 1.4533093e-07 / node index: 18233
                node_order['9606.ENSP00000344741'], # closest to target with label 1 / predicted value: 0.8364509 / node index: 12257
                node_order['9606.ENSP00000470087'], # closest to target with label 0 / predicted value: 0.3407591 / node index: 1674
                node_order['9606.ENSP00000289989'], # furthest to target with label 1 / predicted value: 0.02816797 / node index: 17397
                node_order['9606.ENSP00000348069'], # SREBF1  / predicted value: 0.55870503 / node index: 6334
                node_order['9606.ENSP00000370695']] # : EXCO1 / predicted value: 0.50875914/ node index: 1883 ]

In [6]:
# get node labels for gene ids

from stringdb_alias import HGNCMapper

mapper = HGNCMapper('../9606.protein.info.v11.5.txt.gz', '../9606.protein.aliases.v11.5.txt.gz')

node_series = mapper.get_hgnc_ids(node_id_labels)

In [7]:
mapper.get_hgnc_ids(['9606.ENSP00000289989', # lowest False Negative / predicted value: 0.0281679704785347 / node index: 17397
                '9606.ENSP00000358777', # highest True Positive / predicted value: 0.9995648264884949 / node index: 9005
                '9606.ENSP00000216099', # lowest True Negative / predicted value: 7.5380116e-30 / node index: 8665
                '9606.ENSP00000357637',  # highest False Positive / predicted value: 0.9998927 / node index: 12633
                '9606.ENSP00000357226', # furthest to target with label 0 / predicted value: 1.4533093e-07 / node index: 18233
                '9606.ENSP00000344741', # closest to target with label 1 / predicted value: 0.8364509 / node index: 12257
                '9606.ENSP00000470087', # closest to target with label 0 / predicted value: 0.3407591 / node index: 1674
                '9606.ENSP00000289989', # furthest to target with label 1 / predicted value: 0.02816797 / node index: 17397
                '9606.ENSP00000348069', # SREBF1  / predicted value: 0.55870503 / node index: 6334
                '9606.ENSP00000370695'])

0            C8orf58
1            ATP6AP1
2           APOBEC3D
3               EBF3
4            C1orf61
5             INSIG1
6    ENSG00000268643
7            C8orf58
8             SREBF1
9              EXOC1
dtype: object

In [ ]:
k_vals = [10, 20, 40, 80]

In [9]:
def pos_fidelity_score(fids):
    """
    Takes in list of fidelity scores across multiple instances, and returns overall fidelity score at the model level
    """
    return 1 - sum(fids)/len(fids)

def neg_fidelity_score(fids):
    return sum(fids) / len(fids)

# Random Baseline

Using the DummyExplainer, we will be setting that as a random baseline for explanations. 



In [ ]:
dummy_explainer = Explainer(
            model=model,
            algorithm=DummyExplainer(epochs=200),
            explanation_type='model',
            node_mask_type='attributes', 
            edge_mask_type=None, 
            threshold_config=dict(
                threshold_type='topk', 
                value=k 
            ),
            model_config=dict(
                mode='multiclass_classification',
                task_level='node',
                return_type='log_probs',  
            ),
        )

# Importance of Node Features

In this experiment, the goal is to assess the importance of node features alone by setting the edge mask to None. We will then measure fidelity across different values of k.  

In [12]:
def GNN_attr_feature_experiment(k_vals, test_nodes):
    # measure fidelity across k without edge mask
    wall_times = []
    process_times = []
    explanations = {k:[] for k in k_vals}
    fid_pos_overall = []
    fid_neg_overall = []

    for k in k_vals:
        explainer = Explainer(
            model=model,
            algorithm=GNNExplainer(epochs=200),
            explanation_type='model',
            node_mask_type='attributes',
            edge_mask_type='object',
            threshold_config=dict(
                threshold_type="topk",
                value=k
            ),
            model_config=dict(
                mode='binary_classification',
                task_level='node',
                return_type='probs',
            ),
        )
        fid_pos = []
        fid_neg = []
        for node in test_nodes:
            start = time.time()
            process_start = time.process_time()
            explanation = explainer(data.x, data.edge_index, index = node)
            process_end = time.process_time()
            end = time.time()
            fid = fidelity(explainer, explanation)
            fid_pos.append(fid[0])
            fid_neg.append(fid[1])
            wall_times.append(end - start)
            process_times.append(process_end - process_start)
            explanations[k].append(explanation)
            # explanation.visualize_subgraph(path=f"trial_figs/{explainer}_{node}.pdf", backend='graphviz')
        fid_pos_overall.append(pos_fidelity_score(fid_pos))
        fid_neg_overall.append(neg_fidelity_score(fid_neg))
        

    return {'positive_fid' : fid_pos_overall, 'negative_fid': fid_neg_overall, 'wall_times': wall_times, 'process_times': process_times, 'explanations': explanations}

In [ ]:
gnn_attr_node_importance = GNN_attr_feature_experiment(k_vals, test_nodes)

In [14]:
def GNN_comm_feature_experiment(k_vals, test_nodes):
    # measure fidelity across k without edge mask
    wall_times = []
    process_times = []
    explanations = {k:[] for k in k_vals}
    fid_pos_overall = []
    fid_neg_overall = []

    for k in k_vals:
        explainer = Explainer(
            model=model,
            algorithm=GNNExplainer(epochs=200),
            explanation_type='model',
            node_mask_type='common_attributes',
            edge_mask_type='object',
            threshold_config=dict(
                threshold_type="topk",
                value=k
            ),
            model_config=dict(
                mode='binary_classification',
                task_level='node',
                return_type='probs',
            ),
        )
        fid_pos = []
        fid_neg = []
        for node in test_nodes:
            start = time.time()
            process_start = time.process_time()
            explanation = explainer(data.x, data.edge_index, index = node)
            process_end = time.process_time()
            end = time.time()
            fid = fidelity(explainer, explanation)
            fid_pos.append(fid[0])
            fid_neg.append(fid[1])
            wall_times.append(end - start)
            process_times.append(process_end - process_start)
            explanations[k].append(explanation)
            # explanation.visualize_subgraph(path=f"trial_figs/{explainer}_{node}.pdf", backend='graphviz')
        fid_pos_overall.append(pos_fidelity_score(fid_pos))
        fid_neg_overall.append(neg_fidelity_score(fid_neg))
        

    return {'positive_fid' : fid_pos_overall, 'negative_fid': fid_neg_overall, 'wall_times': wall_times, 'process_times': process_times, 'explanations': explanations}

In [ ]:
gnn_comm_node_importance = GNN_comm_feature_experiment(k_vals, test_nodes)